In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 
from scipy import stats
import seaborn as sns
from scipy import interpolate 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for plots 
# plt.rcParams.update({"text.usetex": True, 'font.size':14})
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# ensuring this notebook generates the same answers 
np.random.seed(1337)

In [ ]:
R = 100000
n = 10
np.random.seed(1337)
F_draw = lambda size: np.random.uniform(0, 1, size)
F_cdf = lambda x: stats.uniform.cdf(x, loc=0, scale=1)
F_pdf = lambda x: stats.uniform.pdf(x, loc=0, scale=1)

In [ ]:
xx = np.linspace(0,1,100)
yy = F_pdf(xx)
vv = F_draw(10_000)
plt.hist(vv, density=True);
plt.plot(xx,yy);

In [ ]:
def draw_from_truncated(F_draw, vbar, n, R): 
    assert vbar > 0.0, f'Cannot draw from a truncated distribution with vbar={vbar}: no observations'
    desired_size = n*R

    # how many extra draws are required? 
    extra_draws = 1.0 / F_cdf(vbar) * 1.1 # 10% more just to be sure 
    largenum = np.round(desired_size*extra_draws).astype(int)

    # draw 
    v_untrunc = F_draw(size=(largenum,))

    # perform truncation
    v_trunc = v_untrunc[(v_untrunc <= vbar)].copy()

    # drop excess rows so we have a regular array
    drop_this_many = np.mod(v_trunc.size, n)
    if drop_this_many != 0: 
        v_trunc = v_trunc[:-drop_this_many]

    # regularize array
    v_sim = v_trunc.reshape(n, -1)

    return v_sim 

In [ ]:
# raw 
v = F_draw(size=(n*R,))
plt.hist(v.flatten(), bins=200, density=True, label='Raw');

# truncated
vbar = 0.8
vsim = draw_from_truncated(F_draw, vbar, n=n, R=R)
plt.hist(vsim.flatten(), bins=20, density=True, label=f'Truncated at {vbar}', alpha=0.4);

plt.xlim(0,1);
plt.legend();


In [ ]:
v = F_draw(size=(n*R,))
# truncated
vbar = 0.8
vsim = draw_from_truncated(F_draw, vbar, n=n, R=R)

fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Histogram(x=v.flatten(),nbinsx=200, name='Raw', histnorm='probability density', opacity=0.6))
fig.add_trace(go.Histogram(x=vsim.flatten(), nbinsx=200, name=f'Truncated at {vbar}', histnorm='probability density', opacity=0.4))
fig.update_layout(title='Histogram of Raw and Truncated Uniform Distribution Samples', xaxis=dict(title='Value', range=[0, 1]), yaxis=dict(title='Density'), barmode='overlay', legend=dict(x=0.8, y=-0.1, orientation="h"), title_x=0.5 )
fig.show()


In [ ]:
def b_star(v): 
    if v == 0.0: 
        return 0.0
    v_losers = draw_from_truncated(F_draw, vbar=v, n=n-1, R=R)
    vmax = v_losers.max(axis=0)
    Ev_2nd = np.mean(vmax)
    return Ev_2nd

In [ ]:
b_star(1.0)

In [ ]:
v = 5.1
bid_shading = b_star(v)/v
print(f'Bidders should bid {bid_shading: 5.2%} of their valuation at {v=: 5.2f}')

In [ ]:
xx = np.linspace(0, 1, 100)
yy = np.array([b_star(v) for v in xx])
plt.plot(xx, yy, '-o', label='$b^*(v)$');
plt.legend();

In [ ]:
xx = np.linspace(0.05, 1, 100)
yy = np.array([b_star(v)/v*100.0 for v in xx])
plt.plot(xx, yy, '-o');
plt.ylabel('FPSB bid in % of valuation');